In [21]:
import pandas as pd
df=pd.read_csv('/kaggle/input/emotion-dataset/df_train_with_explanation.csv')

In [22]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [23]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
    
    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit", # or choose "unsloth/Llama-3.2-1B"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [24]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [27]:
df

,Unnamed: 0,id,text,Anger,Fear,Joy,Sadness,Surprise,generated_explanation
0,0,eng_train_track_a_00001,But not very happy.,0,0,1,1,0,The speaker conveys a sense of dissatisfaction...
1,1,eng_train_track_a_00002,Well she's not gon na last the whole song like...,0,0,1,0,0,The speaker describes a moment of physical sup...
2,2,eng_train_track_a_00003,She sat at her Papa's recliner sofa only to mo...,0,0,0,0,0,The speaker describes a moment of comfort or c...
3,3,eng_train_track_a_00004,"Yes, the Oklahoma city bombing.",1,1,0,1,1,The speaker references a significant historica...
4,4,eng_train_track_a_00005,They were dancing to Bolero.,0,0,1,0,0,The mention of a specific song evokes a sense ...
...,...,...,...,...,...,...,...,...,...
2763,2763,eng_train_track_a_02764,"""Yeah, but did you just find that?",0,1,0,0,1,"The speaker expresses surprise or curiosity, p..."
2764,2764,eng_train_track_a_02765,I did as little as possible with my right hand...,0,0,0,0,0,The speaker describes self-care after an injur...
2765,2765,eng_train_track_a_02766,"Okay that sucks, right?",1,0,0,1,0,"The speaker acknowledges a negative situation,..."
2766,2766,eng_train_track_a_02767,"The spark leaped through his body into mine, a...",0,1,0,0,1,"The speaker describes a sudden, intense connec..."


In [28]:
import pandas as pd
from datasets import Dataset
import json

# Example DataFrame assuming it contains the necessary columns
# df: text, generated_explanation, Anger, Fear, Joy, Sadness, Surprise

# Define the instruction for generating emotional responses
instruction = (
    "Read the provided text carefully and predict the emotional response based on the context. For each of the following emotions — anger, fear, joy, sadness, and surprise — determine whether it is present in the text. "
    "of the sentence. The response should be in a JSON format with emotions as keys and binary values (1 for present, 0 for absent). Please ensure that your response accurately reflects the context and emotional tone conveyed by the sentence."
)

# Function to convert the emotion labels to a JSON format
def format_emotions(row):
    # Create a dictionary with emotions as keys and binary values
    emotions = {
        "Anger": 1 if row['Anger'] == 1 else 0,
        "Fear": 1 if row['Fear'] == 1 else 0,
        "Joy": 1 if row['Joy'] == 1 else 0,
        "Sadness": 1 if row['Sadness'] == 1 else 0,
        "Surprise": 1 if row['Surprise'] == 1 else 0
    }
    
    # Return the JSON formatted string
    return json.dumps(emotions)

# Apply the function to create emotion labels
df['formatted_emotion'] = df.apply(format_emotions, axis=1)

# Prepare the dataset for fine-tuning with the appropriate prompt format
alpaca_prompt = """
### Instruction:
{}
### Input:
{}
### Response:
{}"""

EOS_TOKEN = "<|endoftext|>"  # Replace with the tokenizer's EOS token if necessary

# Create the prompts
df["formatted_prompt"] = df.apply(
    lambda row: alpaca_prompt.format(
        instruction, row["text"],  row["formatted_emotion"]
    ) + EOS_TOKEN, axis=1
)

# Convert to Hugging Face dataset
dataset = Dataset.from_pandas(df[["formatted_prompt"]])

# The dataset is now ready for fine-tuning


In [ ]:
dataset['formatted_prompt']

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [30]:
!export WANDB_MODE=disabled

In [31]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "formatted_prompt",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to='none',
    ),
)


Converting train dataset to ChatML (num_proc=2):   0%|          | 0/2768 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/2768 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/2768 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/2768 [00:00<?, ? examples/s]

In [32]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
12.455 GB of memory reserved.


In [33]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,768 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.258200
2,2.243200
3,2.192500
4,2.019400
5,1.699000
6,1.487500
7,1.156500
8,0.910600
9,0.636900
10,0.551600


In [34]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

201.1714 seconds used for training.
3.35 minutes used for training.
Peak reserved memory = 13.072 GB.
Peak reserved memory for training = 0.617 GB.
Peak reserved memory % of max memory = 88.678 %.
Peak reserved memory for training % of max memory = 4.186 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [35]:
model.save_pretrained("lamma3_model_emotion_detection") # Local saving
tokenizer.save_pretrained("lamma3_model_emotion_detection")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lamma3_model_emotion_detection/tokenizer_config.json',
 'lamma3_model_emotion_detection/special_tokens_map.json',
 'lamma3_model_emotion_detection/tokenizer.json')

In [36]:
# from unsloth import FastLanguageModel
# import torch

# # Model parameters
# model_name = "/kaggle/working/lamma3_model_emotion_detection"  # Update with your model's name
# from unsloth import FastLanguageModel
# model, tokenizer = FastLanguageModel.from_pretrained(
#         model_name = model_name, # YOUR MODEL YOU USED FOR TRAINING
#         max_seq_length = max_seq_length,
#         dtype = dtype,
#         load_in_4bit = load_in_4bit,
#     )
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [37]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import torch

# Define the instruction for generating emotional responses
instruction = (
    "Read the provided text carefully and predict the emotional response based on the context. For each of the following emotions — anger, fear, joy, sadness, and surprise — determine whether it is present in the text. "
    "of the sentence. The response should be in a JSON format with emotions as keys and binary values (1 for present, 0 for absent). Please ensure that your response accurately reflects the context and emotional tone conveyed by the sentence."
)

# Define the Alpaca-style prompt
alpaca_prompt = """
### Instruction:
{}
### Input:
{}
### Response:
"""

# Load your test data from a CSV file
df_test = pd.read_csv('/kaggle/input/emotion-dataset/df_test_with_explanation.csv')
df_test=df_test
# Custom dataset to handle input prompts
class SentenceDataset(Dataset):
    def __init__(self, dataframe):
        self.texts = dataframe["text"].tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return alpaca_prompt.format(instruction, self.texts[idx])


# Create the dataset and dataloader
batch_size = 16
dataset = SentenceDataset(df_test)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Function to generate responses for a batch (predicting emotion labels)
def generate_responses_batch(batch):
    # Tokenize the batch
    inputs = tokenizer(
        batch,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512,
    ).to("cuda")

    # Disable gradient computation for faster inference
    with torch.no_grad():
        # Generate outputs
        outputs = model.generate(  # Directly use the model without DataParallel
            **inputs,
            max_new_tokens=150,
            use_cache=True,
        )

    # Decode the generated outputs
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Extract the predicted emotions
    responses = []
    for decoded_output in decoded_outputs:
        response_start = decoded_output.find("### Response:") + len("### Response:")
        response_end = decoded_output.find("<|endoftext|>")
        response = decoded_output[response_start:response_end].strip()
        responses.append(response)

    return responses

# Generate responses and update the DataFrame
generated_responses = []
for batch in tqdm(dataloader, desc="Generating Responses"):
    responses = generate_responses_batch(batch)
    generated_responses.extend(responses)

# Add generated responses (predicted emotions) to the DataFrame
df_test["predicted_emotions"] = generated_responses

# Print the resulting DataFrame with predicted emotions
# print(df_test[["text", "predicted_emotions"]])


In [62]:
df_test.to_csv('Eng_test_predicted_lamma3.csv',index=False)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import json
import numpy as np

# Helper function to parse the JSON format and convert to binary vector
def parse_json_emotions(emotion_json):
    # Convert JSON string to a dictionary
    emotion_dict = json.loads(emotion_json)
    # Create a binary vector corresponding to emotions [Anger, Fear, Joy, Sadness, Surprise]
    emotions_order = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
    return [emotion_dict.get(emotion, 0) for emotion in emotions_order]

# Assuming actual emotion columns in the test data (Anger, Fear, Joy, Sadness, Surprise)
# Prepare the ground truth (actual emotion columns)
y_true = df_test[['anger', 'fear', 'joy', 'sadness', 'surprise']].values

# Prepare the predictions: convert predicted_emotions from JSON to binary vector format
y_pred = []
for emotion_json in df_test['predicted_emotions']:
    pred_vector = parse_json_emotions(emotion_json)
    y_pred.append(pred_vector)

y_pred = np.array(y_pred)

# Calculate evaluation metrics: Precision, Recall, F1, and Accuracy for each emotion
precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred, average=None)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
for i, label in enumerate(['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']):
    print(f"Emotion: {label}")
    print(f"  Precision: {precision[i]:.4f}")
    print(f"  Recall: {recall[i]:.4f}")
    print(f"  F1 Score: {f1[i]:.4f}")


In [47]:
df_test

,Unnamed: 0,id,text,anger,fear,joy,sadness,surprise,generated_explanation,predicted_emotions
0,0,eng_test_track_a_00001,/ o \ So today I went in for a new exam with D...,1,1,0,0,0,"The speaker describes a stressful situation, l...","{""Anger"": 0, ""Fear"": 1, ""Joy"": 0, ""Sadness"": 0..."
1,1,eng_test_track_a_00002,The image I have in my mind is this: a group o...,0,1,0,1,0,The speaker describes a metaphorical emotional...,"{""Anger"": 0, ""Fear"": 1, ""Joy"": 0, ""Sadness"": 1..."
2,2,eng_test_track_a_00003,"I slammed my fist against the door and yelled,...",1,1,0,0,0,The speaker's frustration or desperation promp...,"{""Anger"": 1, ""Fear"": 0, ""Joy"": 0, ""Sadness"": 0..."
3,3,eng_test_track_a_00004,I could not unbend my knees.,0,1,0,0,0,"The speaker describes a physical limitation, p...","{""Anger"": 0, ""Fear"": 1, ""Joy"": 0, ""Sadness"": 0..."
4,4,eng_test_track_a_00005,"I spent the night at the hotel, mostly hanging...",0,1,0,1,0,The speaker describes a situation of discomfor...,"{""Anger"": 0, ""Fear"": 1, ""Joy"": 0, ""Sadness"": 1..."
...,...,...,...,...,...,...,...,...,...,...
495,495,eng_test_track_a_00496,The guy superficially looked like me (short br...,0,1,0,0,1,The speaker reflects on a superficial resembla...,"{""Anger"": 0, ""Fear"": 0, ""Joy"": 0, ""Sadness"": 0..."
496,496,eng_test_track_a_00497,"I wanted to open my eyes, to make the bad drea...",0,1,0,1,0,The speaker describes a vivid and unpleasant d...,"{""Anger"": 0, ""Fear"": 1, ""Joy"": 0, ""Sadness"": 1..."
497,497,eng_test_track_a_00498,"I got bored and quickly shifted to Windows XP,...",0,0,1,0,0,The speaker describes a moment of relief or es...,"{""Anger"": 0, ""Fear"": 0, ""Joy"": 1, ""Sadness"": 0..."
498,498,eng_test_track_a_00499,after drinking all night i couldnt manage a fa...,0,1,0,1,0,The speaker describes a moment of intense phys...,"{""Anger"": 0, ""Fear"": 1, ""Joy"": 0, ""Sadness"": 1..."
